# T1113 - Screen Capture
Adversaries may attempt to take screen captures of the desktop to gather information over the course of an operation. Screen capturing functionality may be included as a feature of a remote access tool used in post-compromise operations. Taking a screenshot is also typically possible through native utilities or API calls, such as <code>CopyFromScreen</code>, <code>xwd</code>, or <code>screencapture</code>.(Citation: CopyFromScreen .NET)(Citation: Antiquated Mac Malware)


## Atomic Tests

### Atomic Test #1 - Screencapture
Use screencapture command to collect a full desktop screenshot

**Supported Platforms:** macos
#### Attack Commands: Run with `bash`
```bash
screencapture /tmp/T1113_desktop.png
```

In [ ]:
Invoke-AtomicTest T1113 -TestNumbers 1

### Atomic Test #2 - Screencapture (silent)
Use screencapture command to collect a full desktop screenshot

**Supported Platforms:** macos
#### Attack Commands: Run with `bash`
```bash
screencapture -x /tmp/T1113_desktop.png
```

In [ ]:
Invoke-AtomicTest T1113 -TestNumbers 2

### Atomic Test #3 - X Windows Capture
Use xwd command to collect a full desktop screenshot and review file with xwud

**Supported Platforms:** linux
#### Dependencies:  Run with `bash`!
##### Description: Package with XWD and XWUD must exist on device

##### Check Prereq Commands:
```bash
if rpm -q xorg-x11-apps > /dev/null; then exit 0; else exit 1; fi

```
##### Get Prereq Commands:
```bash
sudo yum install -y xorg-x11-apps 

```

In [ ]:
Invoke-AtomicTest T1113 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
xwd -root -out /tmp/T1113_desktop.xwd
xwud -in /tmp/T1113_desktop.xwd
```

In [ ]:
Invoke-AtomicTest T1113 -TestNumbers 3

### Atomic Test #4 - Capture Linux Desktop using Import Tool
Use import command from ImageMagick to collect a full desktop screenshot

**Supported Platforms:** linux
#### Dependencies:  Run with `None`!
##### Description: ImageMagick must be installed

##### Check Prereq Commands:
```None
if import -help > /dev/null 2>&1; then exit 0; else exit 1; fi

```
##### Get Prereq Commands:
```None
sudo apt install graphicsmagick-imagemagick-compat

```

In [ ]:
Invoke-AtomicTest T1113 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
import -window root /tmp/T1113_desktop.png
```

In [ ]:
Invoke-AtomicTest T1113 -TestNumbers 4

### Atomic Test #5 - Windows Screencapture
Use Psr.exe binary to collect screenshots of user display. Test will do left mouse click to simulate user behaviour

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
cmd /c start /b psr.exe /start /output c:\temp\T1113_desktop.zip /sc 1 /gui 0 /stopevent 12
Add-Type -MemberDefinition '[DllImport("user32.dll")] public static extern void mouse_event(int flags, int dx, int dy, int cButtons, int info);' -Name U32 -Namespace W;
[W.U32]::mouse_event(0x02 -bor 0x04 -bor 0x01, 0, 0, 0, 0);
cmd /c "timeout 5 > NULL && psr.exe /stop"
```

In [ ]:
Invoke-AtomicTest T1113 -TestNumbers 5

### Atomic Test #6 - Windows Screen Capture (CopyFromScreen)
Take a screen capture of the desktop through a call to the [Graphics.CopyFromScreen] .NET API.

[Graphics.CopyFromScreen]: https://docs.microsoft.com/en-us/dotnet/api/system.drawing.graphics.copyfromscreen

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
Add-Type -AssemblyName System.Windows.Forms
$screen = [Windows.Forms.SystemInformation]::VirtualScreen
$bitmap = New-Object Drawing.Bitmap $screen.Width, $screen.Height
$graphic = [Drawing.Graphics]::FromImage($bitmap)
$graphic.CopyFromScreen($screen.Left, $screen.Top, 0, 0, $bitmap.Size)
$bitmap.Save("$env:TEMP\T1113.png")
```

In [ ]:
Invoke-AtomicTest T1113 -TestNumbers 6

## Detection
Monitoring for screen capture behavior will depend on the method used to obtain data from the operating system and write output files. Detection methods could include collecting information from unusual processes using API calls used to obtain image data, and monitoring for image files written to disk. The sensor data may need to be correlated with other events to identify malicious activity, depending on the legitimacy of this behavior within a given network environment.

## Shield Active Defense
### Decoy Content  
 Seed content that can be used to lead an adversary in a specific direction, entice a behavior, etc. 

 Decoy Content is the data used to tell a story to an adversary. This content can be legitimate or synthetic data which is used to reinforce or validate your defensive strategy. Examples of decoy content are files on a storage object, entries in the system registry, system shortcuts, etc.
#### Opportunity
There is an opportunity to feed content to an adversary to influence their behaviors, test their interest in specific topics, or add legitimacy to a system or environment.
#### Use Case
A defender can display decoy content on the screen which may be of interest to an adversary in an attempt to elicit further engagement.
#### Procedures
Create directories and files with names and contents using key words that may be relevant to an adversary to see if they examine or exfiltrate the data.
Seed a file system with content that is of no value to the company but reinforces the legitimacy of the system if viewed by an adversary.